In [ ]:
from pprint import pprint
import random
import pandas as pd
import re
import time
import torch
import torch.nn as nn

In [ ]:
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [ ]:
phrases = df['normalized_text'].tolist()

In [ ]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [ ]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
X.shape

torch.Size([10891, 50])

In [ ]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
t = embeddings(X[0:10])
t.shape

torch.Size([10, 50, 28])

In [ ]:
t.view(-1).shape

torch.Size([15247400])

In [ ]:
t.shape, X[0:10].shape

(torch.Size([10, 50, 28]), torch.Size([10, 50]))

In [ ]:
rnn = torch.nn.RNN(28, 128, batch_first=True)
o, s = rnn(t)
o.shape, s.shape

(torch.Size([10, 50, 128]), torch.Size([1, 10, 128]))

In [ ]:
liner = torch.nn.Linear(128, len(INDEX_TO_CHAR))

In [ ]:
liner(o).shape

torch.Size([10, 50, 28])

In [ ]:
rnn = torch.nn.RNN(28, 128)
o, s = rnn(t)
o.shape, s.shape

(torch.Size([10, 50, 128]), torch.Size([1, 50, 128]))

In [ ]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([10, 50, 128]), torch.Size([1, 50, 128]))

28

In [ ]:
class RNN2(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size, bias=False)
    
    def forward(self, x, h = ''):
        output = torch.zeros(x.shape[0], x.shape[1],  self.hidden_size)

        if h == '':
           h = torch.zeros(x.shape[1],  self.hidden_size)

        for i in range(x.shape[0]):
            elem = x[i,:,:]
            res_tens = self.i2h(elem)
            h = torch.tanh(h + res_tens)
            output[i, :, :] = h
        return output, h

In [ ]:
rnn = RNN2(28, 128)
o, h = rnn.forward(t)
print(o.shape, h.shape)

torch.Size([10, 50, 128]) torch.Size([50, 128])


In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__() # 
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28) 
        self.rnn = RNN2(28, 128)
        self.liner = torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.liner(o)
        return out

In [ ]:
batch_size = 100

In [ ]:
model = Network()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [ ]:
for ep in range(200):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / batch_size)):
        batch = X[i * batch_size:(i + 1) * batch_size]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()
        
        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    if ep%5 == 0:
        print(generate_sentence(model))
    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

hellou the the the the the the the the the the the the 
Epoch 0. Time: 6.601, Train loss: 2.240
Epoch 1. Time: 7.395, Train loss: 2.012
Epoch 2. Time: 6.366, Train loss: 1.934
Epoch 3. Time: 6.773, Train loss: 1.890
Epoch 4. Time: 6.356, Train loss: 1.861
helloure whe whe whe whe whe whe whe whe whe whe whe wh
Epoch 5. Time: 6.903, Train loss: 1.840
Epoch 6. Time: 6.296, Train loss: 1.823
Epoch 7. Time: 6.908, Train loss: 1.809
Epoch 8. Time: 8.504, Train loss: 1.797
Epoch 9. Time: 6.489, Train loss: 1.787
helloure whe whe whe whe whe whe whe whe whe whe whe wh
Epoch 10. Time: 7.020, Train loss: 1.777
Epoch 11. Time: 6.552, Train loss: 1.769
Epoch 12. Time: 6.778, Train loss: 1.761
Epoch 13. Time: 6.174, Train loss: 1.754
Epoch 14. Time: 6.887, Train loss: 1.748
helloure whe whe whe whe whe whe whe whe whe whe whe wh
Epoch 15. Time: 6.303, Train loss: 1.742
Epoch 16. Time: 6.792, Train loss: 1.737
Epoch 17. Time: 6.250, Train loss: 1.732
Epoch 18. Time: 6.854, Train loss: 1.727
Epoch 1

In [ ]:
def generate_sentence(model):
    sent = ['h', 'e', 'l', 'l', 'o']

    x = torch.zeros((1, len(sent)), dtype = int)

    for j, w in enumerate(sent):
        if j> MAX_LEN:
          break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])


    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim = True)
        x = torch.cat([x, w.unsqueeze(0)], axis = 1)
        ww = INDEX_TO_CHAR[w]

        if ww == 'none':
            break
        
        sent.append(ww)

    return ''.join(sent)